Compare simulation with experiment
***

For getting _propagate_n0n2_srw() to work:  
* Need to work with range of fft from lines 465-467

Default: Description
* 0: Auto-Resize (1) or not (0) Before propagation,
* 0: Auto-Resize (1) or not (0) After propagation,
* 1.: Relative Precision for propagation with Auto-Resizing (1. is nominal)
* 0: Allow (1) or not (0) for semi-analytical treatment of the quadratic (leading) phase terms at the propagation
* 0: Do any Resizing on Fourier side, using FFT, (1) or not (0)
* 1: Horizontal Range modification factor at Resizing (1. means no modification)
* 1: Horizontal Resolution modification factor at Resizing (1. means no modification)
* 1: Vertical Range modification factor at Resizing (1. means no modification)
* 1: Vertical Resolution modification factor at Resizing (1. means no modification)
* 0: Optional: Type of wavefront Shift before Resizing (vs which coordinates; to be implemented)
* 0: Optional: New Horizontal wavefront Center position after Shift (to be implemented)
* 0: Optional: New Vertical wavefront Center position after Shift (to be implemented)

For getting _propagate_abcd_lct() to work:  
* Use print(rslct.lct_decomposition(abcd_mat_cryst))
    * Change l_scale until it works...

***
#### Imports

In [ ]:
import array
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal
from rslaser.utils import srwl_uti_data as srwutil
from rsmath import lct as rslct

import scipy.constants as const
from scipy.ndimage import gaussian_filter

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

#### File Data

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')

files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_pump_off.txt'), #_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_pump_off.txt'), #_off_peak_timing.txt'), #_pump_off.txt')
)

***
Initiate laser pulse fron the phase-intensity image pairs of a wavefront censor (WFS) diagnostic:

ccd_name = 'ccd_pump_off.txt'
* This file contains the photon count, perhaps normalized, which we loosely refer to as intensity.

wfs_name = 'wfs_pump_off.txt'
* This file contains the phase of the electric field.

### Pulse Init

In [ ]:
params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = 5,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / math.sqrt(2.),
    sigx_waist  = 1.2e-3 /1.18,
    sigy_waist  = 1.2e-3 /1.18,
)

thisPulse = pulse.LaserPulse(params, files)

# x_max_cut = wfr_exp_init.mesh.xFin
# x_min_cut = wfr_exp_init.mesh.xStart
# y_max_cut = wfr_exp_init.mesh.yFin
# y_min_cut = wfr_exp_init.mesh.yStart

In [ ]:
# Plot initial wavefront
wfr_exp_init = thisPulse.slice[0].wfr

x_exp_init=np.linspace(wfr_exp_init.mesh.xStart,wfr_exp_init.mesh.xFin,wfr_exp_init.mesh.nx)
y_exp_init=np.linspace(wfr_exp_init.mesh.yStart,wfr_exp_init.mesh.yFin,wfr_exp_init.mesh.ny)

e_total = thisPulse.extract_total_2d_elec_fields()
intensity_2D_exp_init = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_exp_init*(1e3), x_exp_init*(1e3), intensity_2D_exp_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (pump_off)')

In [ ]:
# initial phase
phase_exp_init = srwlib.array('d', [0]*wfr_exp_init.mesh.nx*wfr_exp_init.mesh.ny) # "flat" array to take 2D data
srwl.CalcIntFromElecField(phase_exp_init, wfr_exp_init, 0, 4, 3, wfr_exp_init.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping data from flat to 2D array
phase_2d_exp_init = np.array(phase_exp_init).reshape((wfr_exp_init.mesh.nx, wfr_exp_init.mesh.ny), order='C').astype(np.float64)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_exp_init*(1e3), x_exp_init*(1e3), phase_2d_exp_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_title('Phase (initial)')

***
### Crystal Init

In [ ]:
num_slices = 5 #5

crystal_params = PKDict(
        length      = 0.025, # [m]
        nslice      = num_slices,
        l_scale     = 0.05, #abcd_lct value #0.004, 
        n0          = [1.76 for _ in range(num_slices)],
        n2          = [16.0 for _ in range(num_slices)],
        pump_energy = 0.035,  # [J]
        pump_waist  = 1.2e-3 /1.18,  # [m]
)

e_crystal = crystal.Crystal(crystal_params)

***
## Check scale factor and resampling

In [ ]:
# photon_e_ev = laser_pulse.photon_e_ev

##Convert energy to wavelength
hc_ev_um = 1.23984198   # hc [eV*um]
phLambda = hc_ev_um / params.photon_e_ev * 1e-6 # wavelength corresponding to photon_e_ev in meters
print("Wavelength corresponding to %g keV: %g microns" %(params.photon_e_ev * 1e-3, phLambda / 1e-6))

# rescale ABCD matrix with wavelength and scale factor for use in LCT algorithm
A = 0.99765495
B = 1.41975385 * phLambda / (crystal_params.l_scale**2)
C = -0.0023775 / phLambda * (crystal_params.l_scale**2)
D = 0.99896716
abcd_mat_cryst = np.array([[ A,  B  ],
            [ C, D ]])
abcd_decomp = rslct.lct_decomposition(abcd_mat_cryst)
print(abcd_decomp)

In [ ]:
prop_mesh_den_pred = abcd_decomp[2][1] * abcd_decomp[5][1] * np.shape(phase_2d_exp_init)[0]
print('predicted propagated mesh density: %g, %g' %(prop_mesh_den_pred, prop_mesh_den_pred))

***
## Propagate through crystal once

In [ ]:
prop_type = 'gain_calc' #'abcd_lct' #'n0n2_lct' #'n0n2_srw'
gain = 1

LP = pulse.LaserPulse(params, files)
nslices_pulse = len(LP.slice)

n_photons_before = 0
for laser_index_i in np.arange(LP.nslice):
        n_photons_before += np.sum(LP.slice[laser_index_i].n_photons_2d.mesh)

e_crystal = crystal.Crystal(crystal_params)
nslices_crystal = len(e_crystal.slice)

print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
LP = e_crystal.propagate(LP, prop_type, gain)

wfr_sim_final=LP.slice[0].wfr

x_sim_final=np.linspace(wfr_sim_final.mesh.xStart,wfr_sim_final.mesh.xFin,wfr_sim_final.mesh.nx)
y_sim_final=np.linspace(wfr_sim_final.mesh.yStart,wfr_sim_final.mesh.yFin,wfr_sim_final.mesh.ny)

***
#### Post Propagation Plots

In [ ]:
e_total = LP.extract_total_2d_elec_fields()
intens_2d_sim_final = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
    
with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), intens_2d_sim_final, cmap=plt.cm.viridis, shading='auto') #viridis
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (simulation gain)')

# x_sim_plot = x_sim_final[np.logical_and(x_sim_final <= x_max_cut,x_sim_final >= x_min_cut)]
# y_sim_plot = y_sim_final[np.logical_and(y_sim_final <= y_max_cut,y_sim_final >= y_min_cut)]

# intens_2d_sim_final_cut = intens_2d_sim_final[np.logical_and(x_sim_final <= x_max_cut,x_sim_final >= x_min_cut),:]
# intens_2d_sim_final_cut = intens_2d_sim_final_cut[:,np.logical_and(y_sim_final <= y_max_cut,y_sim_final >= y_min_cut)]
   
# with plt.style.context(('seaborn-poster')):
#     fig = plt.figure(figsize=(6,3.6))
#     ax = fig.gca()
#     plt.pcolormesh(y_sim_plot*(1e3), x_sim_plot*(1e3), intens_2d_sim_final_cut, cmap=plt.cm.viridis, shading='auto') #viridis
#     plt.colorbar()
#     ax.set_ylabel(r'Vertical Position [mm]')
#     ax.set_xlabel(r'Horizontal Position [mm]')
#     ax.set_title('Intensity (simulation gain)')

In [ ]:
# transmitted pulse - phase
phase_sim_final = srwlib.array('d', [0]*wfr_sim_final.mesh.nx*wfr_sim_final.mesh.ny) # "flat" array to take 2D phase data
srwl.CalcIntFromElecField(phase_sim_final, wfr_sim_final, 0, 4, 3, wfr_sim_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping data from flat to 2D array
phase_2d_sim_final = np.array(phase_sim_final).reshape((wfr_sim_final.mesh.nx, wfr_sim_final.mesh.ny), order='C').astype(np.float64)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), np.unwrap(phase_2d_sim_final), cmap=plt.cm.viridis, shading='auto') #np.unwrap(phase_2d_sim_final)
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')#[mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')#[mm]')
    ax.set_title('Phase (simulation gain)')

# phase_2d_sim_final_cut = phase_2d_sim_final[np.logical_and(x_sim_final <= x_max_cut,x_sim_final >= x_min_cut),:]
# phase_2d_sim_final_cut = phase_2d_sim_final_cut[:,np.logical_and(y_sim_final <= y_max_cut,y_sim_final >= y_min_cut)]
   
# with plt.style.context(('seaborn-poster')):
#     fig = plt.figure(figsize=(6,3.6))
#     ax = fig.gca()
#     plt.pcolormesh(y_sim_plot*(1e3), x_sim_plot*(1e3), np.unwrap(phase_2d_sim_final_cut), cmap=plt.cm.viridis, shading='auto') #viridis
#     plt.colorbar()
#     ax.set_ylabel(r'Vertical Position [mm]')
#     ax.set_xlabel(r'Horizontal Position [mm]')
#     ax.set_title('Phase (simulation post-prop)')


***
## Compare meshes

In [ ]:
# compare initial and final phase mesh density and range
print('Phase mesh analysis:')
print('initial mesh shape: %s , %s' %(np.shape(phase_2d_exp_init)[0], np.shape(phase_2d_exp_init)[1]))
print('initial horizontal range: %g [mm]' %(np.max(x_exp_init)*(1e3) - np.min(x_exp_init)*(1e3)))
print('initial vertical range: %g [mm]' %(np.max(y_exp_init)*(1e3) - np.min(y_exp_init)*(1e3)))
print('final phase mesh shape: %s, %s' %(np.shape(phase_2d_sim_final)[0], np.shape(phase_2d_sim_final)[1]))
print('final horizontal range: %g [mm]' %(np.max(x_sim_final)*(1e3) - np.min(x_sim_final)*(1e3)))
print('final vertical range: %g [mm]' %(np.max(y_sim_final)*(1e3) - np.min(y_sim_final)*(1e3)))

# Plot of Intensity Gain

In [ ]:
# Plot difference in intensities

diff_intensity_2D_sim = intens_2d_sim_final - intensity_2D_exp_init

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), diff_intensity_2D_sim, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (sim difference)')

In [ ]:
print(np.max(intens_2d_sim_final)-np.max(intensity_2D_exp_init))

In [ ]:
prop_n_photons = 0
for laser_index_i in np.arange(LP.nslice):
        prop_n_photons += np.sum(LP.slice[laser_index_i].n_photons_2d.mesh)

print(prop_n_photons -n_photons_before)

***
# Plots of peak timing and off-peak timing (assuming same laser params as pump_off case)

In [ ]:
files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_peak_timing.txt'), #_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_peak_timing.txt'), #_off_peak_timing.txt'), #_pump_off.txt')
)

peakPulse = pulse.LaserPulse(params, files)
wfr_exp_final = peakPulse.slice[0].wfr

x_exp_final=np.linspace(wfr_exp_final.mesh.xStart,wfr_exp_final.mesh.xFin,wfr_exp_final.mesh.nx)
y_exp_final=np.linspace(wfr_exp_final.mesh.yStart,wfr_exp_final.mesh.yFin,wfr_exp_final.mesh.ny)

In [ ]:
e_total = peakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (peak timing)')

In [ ]:
# # Extract phase from experimental post-prop wavefront
# phase_exp_final = srwlib.array('d', [0]*wfr_exp_final.mesh.nx*wfr_exp_final.mesh.ny) # "flat" array to take 2D data
# srwl.CalcIntFromElecField(phase_exp_final, wfr_exp_final, 0, 4, 3, wfr_exp_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# # Reshaping phase data from flat to 2D array
# phase_2d_exp_final = np.array(phase_exp_final).reshape((wfr_exp_final.mesh.nx, wfr_exp_final.mesh.ny), order='C').astype(np.float64)

# with plt.style.context(('seaborn-poster')):
#     fig = plt.figure(figsize=(6,3.6))
#     ax = fig.gca()
#     plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), phase_2d_exp_final, cmap=plt.cm.viridis, shading='auto')
#     plt.colorbar()
#     ax.set_xlabel(r'Horizontal Position [mm]')
#     ax.set_ylabel(r'Vertical Position [mm]')
#     ax.set_title('Phase (experimental peak timing)')

________
With gain on, n1 = 1, n2 = 0, should see gain in peak intensity (gain) that matches the difference between two maximum intensities for peak and off peak timing

In [ ]:
files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_off_peak_timing.txt'), #_pump_off.txt')
)

offpeakPulse = pulse.LaserPulse(params, files)
wfr_exp_final_op = offpeakPulse.slice[0].wfr

# _op = _off_peak

x_exp_final_op=np.linspace(wfr_exp_final_op.mesh.xStart,wfr_exp_final_op.mesh.xFin,wfr_exp_final_op.mesh.nx)
y_exp_final_op=np.linspace(wfr_exp_final_op.mesh.yStart,wfr_exp_final_op.mesh.yFin,wfr_exp_final_op.mesh.ny)

In [ ]:
e_total = offpeakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final_op = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final_op*(1e3), x_exp_final_op*(1e3), intensity_2D_exp_final_op, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (off-peak timing)')

In [ ]:
# Plot difference in intensities

diff_intensity_2D_exp_final = intensity_2D_exp_final - intensity_2D_exp_final_op

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), diff_intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (data difference)')

In [ ]:
print(np.max(intensity_2D_exp_final) - np.max(intensity_2D_exp_final_op))

In [ ]:
peak_n_photons = 0
for laser_index_i in np.arange(peakPulse.nslice):
        peak_n_photons += np.sum(peakPulse.slice[laser_index_i].n_photons_2d.mesh)

offpeak_n_photons = 0
for laser_index_i in np.arange(offpeakPulse.nslice):
        offpeak_n_photons += np.sum(offpeakPulse.slice[laser_index_i].n_photons_2d.mesh)

print(peak_n_photons -offpeak_n_photons)